In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_selection import chi2, f_regression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, plot_roc_curve,precision_score, roc_auc_score
from xgboost import XGBClassifier
from imblearn.over_sampling import ADASYN, SMOTE
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [2]:
#Load data
df=pd.read_csv('application_data.csv', index_col = 0, na_values= ['XAP', 'XNA', 365243])
df_p=pd.read_csv('previous_application.csv', index_col = 0, na_values= ['XAP', 'XNA', 365243])

C:\Users\cibea\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
#df['AGE']=df['DAYS_BIRTH']/(-365)
#age_group=[]

#for i in df.AGE:
    #if i >= 20 and i < 30:
        #age_group.append("20 - 29")
    #if i >= 30 and i < 40:
        #age_group.append("30 - 39")
    #if i >= 40 and i < 50:
        #age_group.append("40 - 49")
    #if i >= 50 and i < 60:
        #age_group.append("50 - 59")
    #if i >= 60 and i < 70:
        #age_group.append("60 - 69")
        
#df['AGE_GROUP']=age_group

In [3]:
def objects(dataframe):
    objectlist = dataframe.dtypes[dataframe.dtypes == 'object'].index
    return objectlist

def numbers(dataframe):
    objectlist = dataframe.dtypes[dataframe.dtypes != 'object'].index
    return objectlist

def integers(dataframe):
    objectlist = dataframe.dtypes[dataframe.dtypes == 'int64'].index
    return objectlist

def floats(dataframe):
    objectlist = dataframe.dtypes[dataframe.dtypes == 'float64'].index
    return objectlist

In [4]:
c=pd.DataFrame()
Name=[]
Percent=[]

for i in df:
    Name.append(df[i].name)
    Percent.append((len(df[i].dropna()) / len(df[i])))
c['Name'] = Name
c['Percent'] = Percent
keep = c[c.Percent > 0.8].Name

In [5]:
X = df[keep[1:]]
y = df[keep[0]]

In [6]:
X_num = X[numbers(X)]
X_cat = X[objects(X)]

In [7]:
boolean = X_num.columns[X_num.max()==1]
cont = X_num.columns[X_num.max()!=1]

In [8]:
X_bool = X_num[boolean]
X_cont = X_num[cont]

In [9]:
## Fill Nan values

for i in X_cat:
    new = X_cat[i].fillna('NA')
    X_cat[i] = new

for i in integers(X_cont):
    new = X_cont[i].fillna(np.median(X_cont[i]))
    X_cont[i] = new
    
for i in integers(X_bool):
    new = X_bool[i].fillna(np.median(X_bool[i]))
    X_bool[i] = new
    
for i in floats(X_cont):
    new = X_cont[i].fillna(np.mean(X_cont[i]))
    X_cont[i] = new

<ipython-input-9-63afee209b8e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_cat[i] = new
<ipython-input-9-63afee209b8e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_cont[i] = new
<ipython-input-9-63afee209b8e>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [10]:
standardscaler = StandardScaler()
ss_X_cont = pd.DataFrame(standardscaler.fit_transform(X_cont), columns=X_cont.columns, index=X_cont.index)

In [11]:
ohe = OneHotEncoder(sparse=False)
X_ohe = ohe.fit_transform(X_cat)
features = ohe.get_feature_names(X_cat.columns)
X_enc=pd.DataFrame(X_ohe, columns=features, index=X_cat.index)
frames = [X_enc, ss_X_cont,X_bool,y]
#X = pd.merge(pd.merge(X_enc, ss_X_cont, how = 'left', left_index=True, right_index=True), X_bool, how = 'left', left_index=True, right_index=True)
df_cleanest=pd.concat(frames, axis=1)
df_cleanest

,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_NA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,...,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,TARGET
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,1
100003.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
100004.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
100006.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
100007.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
456252.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
456253.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
df_cleanest.to_csv('application_data_cleanest.csv')